In [1]:
import argparse
import os

import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets
import torchvision.transforms as transforms
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.backends.cudnn as cudnn
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
from PIL import Image

file_path = r"C:\Users\Essakine\Downloads\CT\mva-recvis-2024\sketch_recvis2024\sketch_recvis2024"

data_transforms = transforms.Compose(
    [
        transforms.Resize((224, 224)),  # Resize to match model input size
        transforms.ToTensor(),  # Convert to tensor
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),  # Normalize
    ]
)
train_loader = torch.utils.data.DataLoader(
        datasets.ImageFolder(file_path + "/train_images", transform=data_transforms),
        batch_size=64,
        shuffle=True,
        num_workers=10,
    )
len(train_loader)

313

In [ ]:
from PIL import Image
import requests
from functools import partial
from transformers import AutoImageProcessor, Swinv2Model
model = Swinv2Model.from_pretrained("microsoft/swinv2-tiny-patch4-window8-256")
mage_processor = AutoImageProcessor.from_pretrained("microsoft/swinv2-tiny-patch4-window8-256")

nclasses = 500



    

class DinoClassifier(nn.Module):
    def __init__(self):
        super(DinoClassifier, self).__init__()
        self.feature_model = torch.hub.load('facebookresearch/dinov2', 'dinov2_vits14').cuda()
        
        # Determine the input features for the classifier
        
        
        self.classifier = nn.Sequential(nn.Linear(768, 256), nn.ReLU(), nn.Linear(256, 500))
    
    def forward(self, x):
        """
        Forward pass through the backbone and the custom classifier.
        """ 
        x = self.feature_model(x)   # Extract features using the DINOv2 backbone
        x = self.feature_model.norm(x)
        output = self.classifier(x)  # Pass features through the classifier
        return output
    
    
class swinv2(nn.Module) :
    def __init__(self) :
        super(swinv2,self).__init__()
        self.featrues_model = Swinv2Model.from_pretrained("microsoft/swinv2-tiny-patch4-window8-256")
        self.image_processor = AutoImageProcessor.from_pretrained("microsoft/swinv2-tiny-patch4-window8-256")
        self.classifier = nn.Sequential(nn.Linear(768, 256), nn.ReLU(), nn.Linear(256, 500))
    def forward(self,x) :
        x = self.image_processor(x, return_tensors="pt")
        x = self.featrues_model(**x)
        x = x.last_hidden_state.squeeze()
        output = self.classifier(x)
        return output


In [13]:

from torch.optim import lr_scheduler

model = DinoClassifier()
model = model.cuda()

# Decay LR by a factor f 0.1 every 7 epochs

    


optimizer = optim.SGD(filter(lambda p: p.requires_grad, model.parameters()), lr=5e-3, momentum=0.9)
exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=6, gamma=0.1)
params = list(model.feature_model.parameters())
total_params = len(params)

# Calculate the index for the first 3/4 of parameters
cutoff_index = int(total_params * 0.75)

# Freeze the first 3/4 parameters
for i, param in enumerate(params):
    if i < cutoff_index:
        param.requires_grad = False
    else:
        param.requires_grad = True
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f"Total parameters: {total_params}")
print(f"Trainable parameters: {trainable_params}")

Using cache found in C:\Users\Essakine/.cache\torch\hub\facebookresearch_dinov2_main


Total parameters: 86905844
Trainable parameters: 21595124


In [14]:
data_path = r"C:\Users\Essakine\Downloads\CT\mva-recvis-2024\sketch_recvis2024\sketch_recvis2024"
experiment = r"C:\Users\Essakine\Downloads\CT\recvis24_a3-main\experiment\New folder"
model_name = 'dinov2'
batch_size = 64
epochs = 10
lr = 5e-3
momentum = 0.9
seed = 1
num_workers = 10
log_interval = 10


In [8]:
def train(
    model: nn.Module,
    optimizer: torch.optim.Optimizer,
    train_loader: torch.utils.data.DataLoader,
    use_cuda: bool,
    epoch: int
    ) -> None:
    """Default Training Loop.

    Args:
        model (nn.Module): Model to train
        optimizer (torch.optimizer): Optimizer to use
        train_loader (torch.utils.data.DataLoader): Training data loader
        use_cuda (bool): Whether to use cuda or not
        epoch (int): Current epoch
        args (argparse.ArgumentParser): Arguments parsed from command line
    """
    model.train()
    correct = 0
    total = 0
    for batch_idx, (data, target) in enumerate(train_loader):
        if use_cuda:
            data, target = data.cuda(), target.cuda()
        optimizer.zero_grad()
        with torch.autocast(device_type='cuda'):
            output = model(data)
            criterion = torch.nn.CrossEntropyLoss(reduction="mean")
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        pred = output.data.max(1, keepdim=True)[1]
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()
        total += target.size(0)
        if batch_idx % log_interval == 0:
            accuracy = 100.0 * correct / total
            print(
                "Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}\tAccuracy: {:.2f}%".format(
                    epoch,
                    batch_idx * len(data),
                    len(train_loader.dataset),
                    100.0 * batch_idx / len(train_loader),
                    loss.data.item(),
                    accuracy,
                )
            )
    print(
        "\nTrain set: Accuracy: {}/{} ({:.0f}%)\n".format(
            correct,
            len(train_loader.dataset),
            100.0 * correct / len(train_loader.dataset),
        )
    )

In [ ]:
def validation(
    model: nn.Module,
    val_loader: torch.utils.data.DataLoader,
    use_cuda: bool,
) -> float:
    """Default Validation Loop.

    Args:
        model (nn.Module): Model to train
        val_loader (torch.utils.data.DataLoader): Validation data loader
        use_cuda (bool): Whether to use cuda or not

    Returns:
        float: Validation loss
    """
    model.eval()
    validation_loss = 0
    correct = 0
    for data, target in val_loader:
        if use_cuda:
            data, target = data.cuda(), target.cuda()
        with torch.autocast(device_type='cuda'):
            output = model(data)
            criterion = torch.nn.CrossEntropyLoss(reduction="mean")
        # sum up batch loss
        validation_loss += criterion(output, target).data.item()
        # get the index of the max log-probability
        pred = output.data.max(1, keepdim=True)[1]
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()

    validation_loss /= len(val_loader.dataset)
    print(
        "\nValidation set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)".format(
            validation_loss,
            correct,
            len(val_loader.dataset),
            100.0 * correct / len(val_loader.dataset),
        )
    )
    return validation_loss

In [15]:
def main():
    """Default Main Function."""
    # options

    # Check if cuda is available
    use_cuda = torch.cuda.is_available()

    # Set the seed (for reproducibility)
    #torch.manual_seed(seed)

    # Create experiment folder
    if not os.path.isdir(experiment):
        os.makedirs(experiment)

    # load model and transform
    if use_cuda:
        print("Using GPU")
        model.cuda()
    else:
        print("Using CPU")

    # Data initialization and loading
    train_loader = torch.utils.data.DataLoader(
        datasets.ImageFolder(data_path + "/train_images", transform=data_transforms),
        batch_size=batch_size,
        shuffle=True,
        num_workers=num_workers,
    )
    val_loader = torch.utils.data.DataLoader(
        datasets.ImageFolder(data_path + "/val_images", transform=data_transforms),
        batch_size=batch_size,
        shuffle=False,
        num_workers=num_workers,
    )

    # Setup optimizer


    # Loop over the epochs
    best_val_loss = 1e8
    for epoch in range(1, epochs + 1):
        # training loop
        train(model, optimizer, train_loader, use_cuda, epoch)
        # validation loop
        val_loss = validation(model, val_loader, use_cuda)
        if val_loss < best_val_loss:
            # save the best model for validation
            best_val_loss = val_loss
            best_model_file = experiment + "/model_best.pth"
            torch.save(model.state_dict(), best_model_file)
        # also save the model every epoch
        model_file = experiment + "/model_" + str(epoch) + ".pth"
        torch.save(model.state_dict(), model_file)
        print(
            "Saved model to "
            + model_file
            + f". You can run `python evaluate.py --model_name {model_name} --model "
            + best_model_file
            + "` to generate the Kaggle formatted csv file\n"
        )

main()


Using GPU
Train Epoch: 1 [0/20000 (0%)]	Loss: 6.367188	Accuracy: 0.00%
Train Epoch: 1 [640/20000 (3%)]	Loss: 6.199219	Accuracy: 0.71%
Train Epoch: 1 [1280/20000 (6%)]	Loss: 5.984375	Accuracy: 1.86%
Train Epoch: 1 [1920/20000 (10%)]	Loss: 5.449219	Accuracy: 3.23%
Train Epoch: 1 [2560/20000 (13%)]	Loss: 5.234375	Accuracy: 5.72%
Train Epoch: 1 [3200/20000 (16%)]	Loss: 4.726562	Accuracy: 8.33%
Train Epoch: 1 [3840/20000 (19%)]	Loss: 3.738281	Accuracy: 10.63%
Train Epoch: 1 [4480/20000 (22%)]	Loss: 3.589844	Accuracy: 13.38%
Train Epoch: 1 [5120/20000 (26%)]	Loss: 3.062500	Accuracy: 16.20%
Train Epoch: 1 [5760/20000 (29%)]	Loss: 2.859375	Accuracy: 18.42%
Train Epoch: 1 [6400/20000 (32%)]	Loss: 2.722656	Accuracy: 20.20%
Train Epoch: 1 [7040/20000 (35%)]	Loss: 2.394531	Accuracy: 22.45%
Train Epoch: 1 [7680/20000 (38%)]	Loss: 2.033203	Accuracy: 24.41%
Train Epoch: 1 [8320/20000 (42%)]	Loss: 1.995117	Accuracy: 26.49%
Train Epoch: 1 [8960/20000 (45%)]	Loss: 2.037109	Accuracy: 28.32%
Train Epoch: 